In [91]:
import pandas as pd
import numpy as np
from datetime import datetime

In [92]:
# Function for Normalization
def normalize(series):
    min_val, max_val = series.min(), series.max()
    return (series - min_val) / (max_val - min_val) if max_val > min_val else 0

In [93]:
#global variables
current_date = datetime.now().strftime("%Y%m%d")
COUNTRY_SCORE_WEIGHTS = {'Exports': 0.57, 'Imports': 0.43, 'Gross_Deals': 0.0}
CATEGORY_SCORE_WEIGHTS = {'Exports': 0.57, 'Imports': 0.43, 'Gross_Deals': 0.0}


# Load the dataset
df = pd.read_csv("final_data/20241122one_row_one_country_data.csv")

In [94]:
# Compute country_metrics
exports = df[df['Country Role'] == 'Supplier'].groupby('Country')['SIPRI TIV for total order'].sum()
imports = df[df['Country Role'] == 'Recipient'].groupby('Country')['SIPRI TIV for total order'].sum().abs()
gross_deals = exports.abs() + imports

# Normalize country_metrics
country_metrics = pd.DataFrame({'Exports': exports, 'Imports': imports, 'Gross_Deals': gross_deals}).fillna(0)
country_metrics['Normalized_Exports'] = normalize(country_metrics['Exports'])
country_metrics['Normalized_Imports'] = normalize(country_metrics['Imports'])
country_metrics['Normalized_Gross'] = normalize(country_metrics['Gross_Deals'])

# Compute composite score

country_metrics['Country_Score'] = round(
    COUNTRY_SCORE_WEIGHTS['Exports'] * country_metrics['Normalized_Exports'] +
    COUNTRY_SCORE_WEIGHTS['Imports'] * country_metrics['Normalized_Imports'] +
    COUNTRY_SCORE_WEIGHTS['Gross_Deals'] * country_metrics['Normalized_Gross']
    , 5
) * 100

# Sort by score
country_metrics = country_metrics.sort_values(by='Country_Score', ascending=False)
display(country_metrics)


,Exports,Imports,Gross_Deals,Normalized_Exports,Normalized_Imports,Normalized_Gross,Country_Score
Country,,,,,,,
United States,752270.38,42010.47,794280.85,1.000000,0.299696,1.000000,69.887
India,1028.96,140176.72,141205.68,0.001368,1.000000,0.177778,43.078
Soviet Union,373140.03,21974.87,395114.90,0.496019,0.156765,0.497450,35.014
Saudi Arabia,170.04,84180.94,84350.98,0.000226,0.600534,0.106198,25.836
Japan,1556.11,75839.67,77395.78,0.002069,0.541029,0.097441,23.382
...,...,...,...,...,...,...,...
PKK (Turkiye)*,0.00,0.72,0.00,0.000000,0.000005,0.000000,0.000
PRC (Israel/Palestine)*,0.00,0.25,0.00,0.000000,0.000002,0.000000,0.000
Provisional IRA (UK)*,0.00,1.20,0.00,0.000000,0.000009,0.000000,0.000


In [95]:
# Initialize an empty dictionary to store category-wise scores
category_scores = {}

# Get the unique categories
categories = df['Category'].unique()

for category in categories:
    # Filter data for the current category
    category_data = df[df['Category'] == category]
    
    # Compute Exports, Imports, and Gross Deals for the current category
    exports = category_data[category_data['Country Role'] == 'Supplier'].groupby('Country')['SIPRI TIV for total order'].sum()
    imports = category_data[category_data['Country Role'] == 'Recipient'].groupby('Country')['SIPRI TIV for total order'].sum().abs()
    gross_deals = exports.abs() + imports
    
    # Normalize category_metrics
    category_metrics = pd.DataFrame({'Exports': exports, 'Imports': imports, 'Gross_Deals': gross_deals}).fillna(0)
    category_metrics['Normalized_Exports'] = normalize(category_metrics['Exports'])
    category_metrics['Normalized_Imports'] = normalize(category_metrics['Imports'])
    category_metrics['Normalized_Gross'] = normalize(category_metrics['Gross_Deals'])
    
    # Define weights and calculate composite score
    category_metrics['Country_Score'] = round(
        CATEGORY_SCORE_WEIGHTS['Exports'] * category_metrics['Normalized_Exports'] +
        CATEGORY_SCORE_WEIGHTS['Imports'] * category_metrics['Normalized_Imports'] +
        CATEGORY_SCORE_WEIGHTS['Gross_Deals'] * category_metrics['Normalized_Gross']
        , 5
    ) * 100
    
    # Store the scores in the dictionary
    category_scores[category] = category_metrics[['Country_Score']]
    
# Combine all category scores into one DataFrame for analysis
final_category_scores = pd.concat(category_scores, axis=1)
final_category_scores.columns = [f"{category}_Score" for category in category_scores.keys()]

# Display the final scores
display(final_category_scores)

,Aircraft_Score,Specialized Equipment_Score,Missiles/Rockets/Bombs_Score,Helicopters_Score,Unmanned Vehicles_Score,Ground Vehicles/Artillery_Score,Radar/Detection Systems_Score,Naval Vessels_Score
Country,,,,,,,,
Afghanistan,3.161,0.160,7.346,9.272,1.114,7.847,0.479,NaN
Albania,0.434,NaN,0.181,0.475,0.371,1.506,NaN,1.271
Algeria,8.638,1.723,8.474,11.095,13.737,15.104,2.943,11.755
Angola,2.714,0.553,2.387,4.952,2.362,2.839,2.888,0.772
Antigua and Barbuda,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032
Solomon Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.064
Timor-Leste,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.115


In [96]:
# Inner join based on 'Country'
result = pd.merge(country_metrics, final_category_scores, on='Country')
result_with_loop = pd.concat([result, final_category_scores['Aircraft_Score'].rename('ZZ_loop_score')], axis=1)
display(result_with_loop)

,Exports,Imports,Gross_Deals,Normalized_Exports,Normalized_Imports,Normalized_Gross,Country_Score,Aircraft_Score,Specialized Equipment_Score,Missiles/Rockets/Bombs_Score,Helicopters_Score,Unmanned Vehicles_Score,Ground Vehicles/Artillery_Score,Radar/Detection Systems_Score,Naval Vessels_Score,ZZ_loop_score
Country,,,,,,,,,,,,,,,,
United States,752270.38,42010.47,794280.85,1.000000,0.299696,1.000000,69.887,64.596,100.000,73.441,69.018,58.355,51.403,70.911,51.301,64.596
India,1028.96,140176.72,141205.68,0.001368,1.000000,0.177778,43.078,43.011,19.533,41.820,22.714,43.000,43.087,43.261,43.520,43.011
Soviet Union,373140.03,21974.87,395114.90,0.496019,0.156765,0.497450,35.014,27.890,2.025,30.787,20.318,31.130,65.129,18.469,42.352,27.890
Saudi Arabia,170.04,84180.94,84350.98,0.000226,0.600534,0.106198,25.836,26.355,6.626,43.000,15.940,17.868,22.339,18.849,12.480,26.355
Japan,1556.11,75839.67,77395.78,0.002069,0.541029,0.097441,23.382,29.353,23.073,31.596,43.078,5.735,4.240,9.594,1.217,29.353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PKK (Turkiye)*,0.00,0.72,0.00,0.000000,0.000005,0.000000,0.000,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,NaN
PRC (Israel/Palestine)*,0.00,0.25,0.00,0.000000,0.000002,0.000000,0.000,NaN,NaN,0.001,NaN,NaN,NaN,NaN,NaN,NaN
Provisional IRA (UK)*,0.00,1.20,0.00,0.000000,0.000009,0.000000,0.000,NaN,NaN,0.003,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
#Exporting the result to a csv file
result_with_loop.to_csv('final_data/'+current_date+'_country_score_data.csv')

In [98]:
result_with_loop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 261 entries, United States to MTA (Myanmar)*
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Exports                          261 non-null    float64
 1   Imports                          261 non-null    float64
 2   Gross_Deals                      261 non-null    float64
 3   Normalized_Exports               261 non-null    float64
 4   Normalized_Imports               261 non-null    float64
 5   Normalized_Gross                 261 non-null    float64
 6   Country_Score                    261 non-null    float64
 7   Aircraft_Score                   194 non-null    float64
 8   Specialized Equipment_Score      156 non-null    float64
 9   Missiles/Rockets/Bombs_Score     193 non-null    float64
 10  Helicopters_Score                183 non-null    float64
 11  Unmanned Vehicles_Score          140 non-null    float64
 12  Grou

In [99]:
result_with_loop.describe()

,Exports,Imports,Gross_Deals,Normalized_Exports,Normalized_Imports,Normalized_Gross,Country_Score,Aircraft_Score,Specialized Equipment_Score,Missiles/Rockets/Bombs_Score,Helicopters_Score,Unmanned Vehicles_Score,Ground Vehicles/Artillery_Score,Radar/Detection Systems_Score,Naval Vessels_Score,ZZ_loop_score
count,261.000000,261.000000,261.000000,2.610000e+02,261.000000,261.000000,261.000000,194.000000,156.000000,193.000000,183.000000,140.000000,205.000000,137.000000,157.000000,194.000000
mean,7654.255632,7654.255632,15069.958123,1.017487e-02,0.054604,0.018973,2.927939,3.455387,3.286641,4.137192,3.761885,4.243121,3.952956,4.762708,5.715140,3.455387
std,54333.239315,16671.941776,59919.886720,7.222568e-02,0.118935,0.075439,7.165584,7.532078,10.208812,9.062180,7.592415,9.313552,8.547459,9.083199,10.433566,7.532078
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001000,0.010000,0.000000
25%,0.000000,30.990000,0.000000,0.000000e+00,0.000221,0.000000,0.011000,0.032750,0.037000,0.008000,0.173500,0.136750,0.049000,0.182000,0.109000,0.032750
50%,0.640000,402.280000,14.260000,8.507579e-07,0.002870,0.000018,0.123000,0.305500,0.214000,0.314000,0.814000,0.645000,0.459000,1.165000,0.830000,0.305500
75%,103.000000,7164.060000,7069.180000,1.369189e-04,0.051107,0.008900,2.223000,3.200000,1.919250,3.345000,3.849000,2.263000,3.353000,4.380000,6.567000,3.200000
max,752270.380000,140176.720000,794280.850000,1.000000e+00,1.000000,1.000000,69.887000,64.596000,100.000000,73.441000,69.018000,58.355000,65.129000,70.911000,61.688000,64.596000


In [100]:
# Melt the result DataFrame
melted_result = result_with_loop.reset_index().melt(
    id_vars=['Country'],  # Columns to keep as identifiers
    var_name='Metric Name',  # Name for the new column with metric names
    value_name='Metric Value'  # Name for the new column with metric values
).sort_values(by=['Country', 'Metric Name'])

# Display the melted DataFrame
display(melted_result)
melted_result.to_csv('final_data/'+current_date+'_country_score_data_melted.csv')

,Country,Metric Name,Metric Value
2066,ANC (South Africa)*,Aircraft_Score,NaN
1805,ANC (South Africa)*,Country_Score,0.000
239,ANC (South Africa)*,Exports,0.000
761,ANC (South Africa)*,Gross_Deals,0.000
3371,ANC (South Africa)*,Ground Vehicles/Artillery_Score,NaN
...,...,...,...
1180,unknown supplier(s),Normalized_Imports,0.000
3529,unknown supplier(s),Radar/Detection Systems_Score,0.003
2224,unknown supplier(s),Specialized Equipment_Score,0.379
3007,unknown supplier(s),Unmanned Vehicles_Score,0.212
